## Import packages

In [ ]:
import ck.kernel as ck
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import json
import os

In [ ]:
print "Collective Knowledge: v%s" % ck.version({})['version_str']
print "pandas: v%s" % pd.version.version
print "NumPy: v%s" % np.version.version
print "Matplotlib: v%s" % matplotlib.__version__
print "JSON: v%s" % json.__version__

In [ ]:
%matplotlib inline

## Find results

In [ ]:
dataset = 'SGEMM_NT'
data_uoa =  dataset + '-explore-f'
module_uoa = 'experiment'

In [ ]:
r=ck.access({'action':'list_points', 'module_uoa':module_uoa, 'data_uoa':data_uoa})
if r['return']>0:
  print ("Error: %s" % r['error'])
  exit(1)

## Show results

### Data frame

In [ ]:
data_list  = []
index_list = []
num_repetitions = 4

# If the 'data' list has fewer than 'num_repetition' elements, 
# extend to have exactly 'num_repetitions' elements with 'ext'.
def extend_to_num_repetitions(data, ext=None, num_repetitions=num_repetitions):
    return data + (num_repetitions - len(data)) * [None]

for point in r['points']:
    with open(os.path.join(r['path'], 'ckp-%s.flat.json' % point)) as point_file:
        point_data = json.load(point_file)
    num_data = 0
    
    # Data.
    Gflops_per_s = point_data.get("##characteristics#run#run_time_state#EXECUTION#Gflops/s#all")
    Gflops_per_s = extend_to_num_repetitions(Gflops_per_s)
    Gflops_per_s_index = num_data
    num_data += 1
    
    diff = point_data.get("##characteristics#run#run_time_state#RESULTS#max_abs_diff#all")
    diff = extend_to_num_repetitions(diff)
    diff_index = num_data
    num_data += 1
    
    match = point_data.get("##characteristics#run#run_time_state#RESULTS#match#all")
    match = extend_to_num_repetitions(match)
    match_index = num_data
    num_data += 1
        
    total_num_data = num_data * num_repetitions
    data_entry = [0] * total_num_data
    data_entry[Gflops_per_s_index:total_num_data:num_data] = Gflops_per_s
    data_entry[match_index:total_num_data:num_data] = match
    data_entry[diff_index:total_num_data:num_data] = diff
    data_list.append(data_entry)
    
    # Index.
    cl_file = point_data.get("##characteristics#run#run_time_state#METADATA#file#all_unique")[0]
    lws_j = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_j#all_unique")[0]
    lws_i = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_i#all_unique")[0]
    local_work_size = ('(%s, %s)' % (lws_j, lws_i))
    matrix_order = point_data.get("##characteristics#run#run_time_state#CMD_LINE_ARGS#matrix_order#all_unique")[0]
    index_list.append((cl_file, local_work_size, matrix_order))

In [ ]:
ci = pd.MultiIndex.from_arrays(names=['Repetition', 'Metric'], arrays=[
        [s for s in range(num_repetitions) for t in range(num_data)], # 0,...,0,1,...,1,2,...,2,...
        ['Gflops/s', 'Diff', 'Match?'] * num_repetitions])
mi = pd.MultiIndex.from_tuples(names=['OpenCL program', 'Local work size', 'Matrix order'] , tuples=index_list)
df = pd.DataFrame(data=data_list, index=mi, columns=ci).sortlevel(level='OpenCL program')

In [ ]:
df

### Plot

In [ ]:
df = df.swaplevel('Repetition', 'Metric', axis=1)['Gflops/s']
df

In [ ]:
mean = df.mean(axis=1)
std = df.std(axis=1)
ymax = np.int64(mean.max() + std.max())

In [ ]:
plot = mean \
    .unstack(level='Local work size').unstack(level='OpenCL program') \
    .plot(yerr=std.unstack(level='Local work size').unstack(level='OpenCL program'), 
        title='Gflops/s vs Matrix order',
        kind='bar', figsize=(12,8), colormap=matplotlib.cm.autumn,
        ylim=(0, ymax), yticks=range(ymax))

In [ ]:
plot.get_figure().savefig('%s-match_tmp.png' % data_uoa)